# Sequence to Sequence Learning with Neural Networks  
- paper review

## Abstract  
- DNN은 sequence에서 sequence로 mapping하는 task를 수행하지 못함  
- 이 논문에서 우리는 general end-to-end sequence to sequence learning을 제안한다  
- phrase-based SMT system은 BLEU score를 33.3을 달성한 반면 LSTM을 사용한 translation task에서 34.8 달성했다  
- 또한 LSTM은 long sentence에서도 학습이 잘 되었다  
- 우리는 source sentence에서 words의 order를 뒤바꿔서 LSTM의 성능을 뚜렷하게 개선시켰다  
- source와 target sentence의 short term dependencies로 인해 opmization problem이 조금 더 쉬워졌다고 가정한다  

## Introduction   
<img src = "https://github.com/Sangh0/NLP/blob/main/Seq2Seq/figures/figure1.png?raw=true">  


- DNN에서는 고정된 dimension의 vector로 encoding하는 문제에 적용할 수 있다  
- 그러나 output의 length를 알 수 없는 sequence to sequence problem에서는 적합하지 않다  
- 그래서 우리는 LSTM architecture를 이용한 application이 sequence to sequence problem을 해결할 수 있음을 보일 것이고 encoder LSTM을 이용해 large fixed dimensional vector representation을 얻고 decoder LSTM을 이용해 output sequence를 출력한다  
- LSTM은 long range temporal dependencies에 대해 잘 학습이 되며 input와 output 사이의 상당한 time lag로 인해 이 application에 적합하다  
- 5개의 deep LSTM의 ensemble로 34.81의 BLEU score를 얻었다  
- long sentences에 대해 학습이 잘 되었는데 그 이유가 source sentence는 reversing하고 target sentence는 reversing하지 않다고 추측하고 있다  
- 결과적으로 SGD를 이용해 학습하면 long sentence에 대한 trouble없이 잘 수렴한다  
- 이 간단한 trick이 이 논문의 핵심이다  


## The model   
- standard RNN은 다음과 같이 계산된다  
$$h_t = sigmoid\left(W^{hx}x_t+W^{hh}h_{t-1}\right)$$  
$$y_t=W^{yh}h_t$$  

- 그러나 RNN은 복잡하고 단조롭지 않은 relationship을 가지는 input과 output에 대해서는 적용이 어려운 문제가 있다  
- 또한 long term dependency에 대해서도 학습이 어려운 점이 있다  
- LSTM은 이를 해결할 수 있는 network이며 우리는 이를 채택한다  
- LSTM의 목표는 conditional probability $p\left(y_1,\cdots,y_{T'}\vert x_1,\cdots,x_T\right)$을 추정하는 것이다  
- 이때 $\left(x_1,\cdots,x_T\right)$은 input sequence이며 $y_1,\cdots,y_{T'}$은 이에 해당하는 output sequence이며 length $T$와 $T'$는 다르다  
- 따라서 standard LSTM-LM formulation은  
$$p\left(y_1,\cdots,y_{T'}\vert x_1,\cdots,x_T\right)=\prod_{t=1}^{T'}p\left(y_t\vert v, y_1,\cdots,y_{t-1}\right)$$  
- 먼저 우리는 두 개의 다른 LSTM을 사용하는데 각각 encoder, decoder를 담당한다  
- 두 번째로, 우리는 deep LSTM은 shallow LSTM을 능가한다는 것을 발견했으며 우리는 4개의 layer로 이루어진 LSTM을 선택한다  
- 마지막으로 input sequence word에서 order를 바꿈으로써 모델의 성능이 개선된 것을 발견했다  
- 예를 들자면 $a, b, c$를 $\alpha, \beta, \gamma$로 mapping한다면 $c, b, a$를 $\alpha, \beta, \gamma$로 mapping하는 것이다

## Experiments  
### Dataset details  
- We used the WMT'14 English to French dataset  
- 12M sentences consisting of 348M French words and 304M English words  

### Decoding and Rescoring  
- 우리는 다음의 log probability를 maximizing한다  
$$\frac{1}{\vert S \vert}\sum_{\left(T, S\right)\in S}log p\left(T\vert S\right)$$  
- 이때 $T$는 correct translation을 의미하고 $S$는 training set을 의미한다  
- 학습이 완료되면 다음과 같은 network의 output에 따라 가장 확률이 높은 값을 찾아 번역한다  
$$\hat{T}=arg max _T p\left(T\vert S\right)$$  
- $<EOS>$가 hypothesis에 추가되는 즉시 beam에서 제거되고 complete hypothesis에 추가된다  

### Reversing the Source Sentences  
- 우리는 reversing을 통해 perplexity가 5.8에서 4.7로 낮아졌고 BLEU score가 25.9에서 30.6으로 개선되었음을 찾았다  
- 이러한 현상에 대해 완벽한 설명을 하기엔 어렵지만 dataset의 short term dependency 때문이라고 가설을 세워본다  
- reversing이 source sentence와 target sentence의 관련된 단어끼리의 average distance에 변화를 주진 못하지만 source sentence의 처음 단어는 target sentence의 처음 단어와 매우 밀접해 minimal time lag은 크게 감소하게 된다  
- 또한 backpropagation 과정에서 establishing communication이 쉬워진다  
- 따라서 성능 향상이 일어났다고 추측한다  

### Training details  
- weight initialization: uniform distribution between $-0.08$ and $0.08$  
- We use SGD without momentum  
- learning rate: 0.7, after 5 epochs, we begun halving the learning rate every half epoch  
- batches of 128  
- LSTM은 vanishing gradient problem에 고통받지 않지만 exploding gradient 문제가 발생한다  
- 따라서 우리는 gradient에 강한 constraint를 가한다  
- $s=\Vert g\Vert_2$, where $g$ is the gradient diviced by 128  
- If $s>5$, we set $g=\frac{5g}{s}$  

### Experimental Results    

<img rsc = "https://github.com/Sangh0/NLP/blob/main/Seq2Seq/figures/table1.png?raw=true">  

<img rsc = "https://github.com/Sangh0/NLP/blob/main/Seq2Seq/figures/table2.png?raw=true">  
  
  
- random initialization과 random order of minibatches를 사용해 LSTM을 ensemble한 것이 가장 성능이 좋았다  

### Model Analysis  
<img rsc = "https://github.com/Sangh0/NLP/blob/main/Seq2Seq/figures/table3.png?raw=true">  
<img rsc = "https://github.com/Sangh0/NLP/blob/main/Seq2Seq/figures/figure3.png?raw=true">    




## Conclusion  
- 우리는 large deep LSTM이 standard SMT-based system보다 성능이 좋다는 것을 보여줬다  
- 또한 충분한 train data가 있다면 다른 많은 sequence problem에서 잘 수행될 것임을 의미한다  
- 가장 핵심적인 부분은 word order를 reversing할 때 성능이 오른다는 것이다  
- 또한 long sentence의 translation 성능이 좋다  
- 단순하고 간단하며 상대적으로 최적화되지 않았음에도 불구하고 SMT system을 능가한다는 것을 보여줬고 여기서 추가 작업을 더 한다면 정확도는 더 오를 것이라 예상한다